In [115]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
#https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure
import six
from pathlib import Path

In [116]:
# Fix region names, remove underscore
def fix_region_name(roi):
    roi = roi.replace('_', ' ')
    roi = ' '.join(roi.split(',')[::-1]).strip()
    return roi

## Insert table path, figures path, weekly timepoints, last week

In [118]:
table_path = Path('/Users/schwartzao/Desktop/workspace/covid-sicr/tables/20210311_realtime/')
weeks = ['13', '30', '47']
weeks_dates = ['April 19-25, 2020', 'August 16-22, 2020', 'December 13-19, 2020', 'March 1-6, 2021'] # FOR COLUMN LABELS

last_week = 58 # use an int
last_week_str = '(March 1-6, 2021)'


df = pd.read_csv(table_path / "fit_table_reweighted.csv") 

df['roi'] = df['roi'].apply(fix_region_name)

rois = list(df.roi.unique())

roi_us = np.sort([i for i in rois if i[:2]=='US'])
roi_other = np.sort([i for i in rois if i[:2]!='US'])

roi_other = list(roi_other) # removing super regions for now
roi_other = [x for x in roi_other if not x.startswith("AA")]

rois = list(roi_other) + list(roi_us)

stats = ['Rt', 'car', 'ifr', 'ir', 'ar']
for stat in stats:
    print(stat)
    df[f'{stat} {last_week_str}'] = 0
    for roi in rois:
        for wk in np.arange(last_week,0,-1):
    #         print(f"{stat} (week {wk})")
            if df.loc[df.roi==roi][f"{stat} (week {wk})"].notnull().values[0]:
                for q in df['quantile'].unique():
                    df.loc[(df.roi==roi)&(df['quantile']==q),[f'{stat} {last_week_str}']] = df.loc[(df.roi==roi)&(df['quantile']==q),[f"{stat} (week {wk})"]].values
    #                 print(df.loc[(df.roi==roi)&(df['quantile']==q)]['Rt (May 24th)'].values)
                break

Rt
car
ifr
ir
ar


In [111]:
theta = [stat + f' (week {week})' for stat in stats for week in weeks]
theta_last = [stat + f' {last_week_str}' for stat in stats]
theta_ = theta + theta_last

theta_R = [x for x in theta_ if x.startswith("R")]
theta_car = [x for x in theta_ if x.startswith("car")]
theta_ifr = [x for x in theta_ if x.startswith("ifr")]
theta_ir = [x for x in theta_ if x.startswith("ir")]
theta_ar = [x for x in theta_ if x.startswith("ar")]
print(theta_)

['Rt (week 13)', 'Rt (week 30)', 'Rt (week 47)', 'car (week 13)', 'car (week 30)', 'car (week 47)', 'ifr (week 13)', 'ifr (week 30)', 'ifr (week 47)', 'ir (week 13)', 'ir (week 30)', 'ir (week 47)', 'ar (week 13)', 'ar (week 30)', 'ar (week 47)', 'Rt (March 1-6, 2021)', 'car (March 1-6, 2021)', 'ifr (March 1-6, 2021)', 'ir (March 1-6, 2021)', 'ar (March 1-6, 2021)']


In [114]:
def afun1(x):
    return '%s' % float('%.1g' %x)

def afun2(x):
    return '%s' % float('%.2g' %x)

ir_col = [f"IRt ({weeks_dates[0]}) (CI)",
          f"IRt ({weeks_dates[1]}) (CI)", 
          f"IRt ({weeks_dates[2]}) (CI)",
          f"IRt ({weeks_dates[3]}) (CI)",
          
         ]

r_col = [f"R0 (CI)",
         f"Rt ({weeks_dates[0]}) (CI)", 
         f"Rt ({weeks_dates[1]}) (CI)",
         f"Rt ({weeks_dates[2]}) (CI)",
         f"Rt ({weeks_dates[3]}) (CI)"
        ]
car_col  = [f"CARt ({weeks_dates[0]}) (CI)", 
          f"CARt ({weeks_dates[1]}) (CI)",
          f"CARt ({weeks_dates[2]}) (CI)",
          f"CARt ({weeks_dates[3]}) (CI)"
         ]

ifr_col = [f"IFRt ({weeks_dates[0]}) (CI)", 
          f"IFRt ({weeks_dates[1]}) (CI)", 
          f"IFRt ({weeks_dates[2]}) (CI)",
          f"IFRt ({weeks_dates[3]}) (CI)"
         ]

ar_col = [f"ARt ({weeks_dates[0]}) (CI)",
          f"ARt ({weeks_dates[1]}) (CI)", 
          f"ARt ({weeks_dates[2]}) (CI)",
          f"ARt ({weeks_dates[3]}) (CI)"
         ]

stats = ['Rt', 'car', 'ifr', 'ir', 'ar']

for stat in stats:
    if stat == 'Rt':
        theta_stat = [x for x in theta_ if x.startswith("R")]
        theta_stat.insert(0, "R0")
        col = r_col

    if stat == 'car':
        theta_stat = [x for x in theta_ if x.startswith("car")]
        col = car_col
    if stat == 'ifr':
        theta_stat = [x for x in theta_ if x.startswith("ifr")]
        col = ifr_col

    if stat == 'ir':
        theta_stat = [x for x in theta_ if x.startswith("ir")]
        col = ir_col

    if stat == 'ar':
        theta_stat = [x for x in theta_ if x.startswith("ar")]
        col = ar_col
    
    rows = []
    for roi in rois:
        data = []
        data.append(roi)
        for i in range(len(theta_stat)):
            theta = theta_stat[i]

            mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
            if theta[0] == 'R':
                x = afun2(mu)+" ("+afun2(lb)+", "+afun2(ub)+")"
            else:
                x = afun1(mu)+" ("+afun1(lb)+", "+afun1(ub)+")"
                
            data.append(x)
        rows.append(data)

        df_report = pd.DataFrame(rows, columns=['Region']+col)
        df_report.to_csv(table_path / f"{stat}_summary.csv", index=False)


In [5]:
# def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
#                      header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
#                      bbox=[0, 0, 1, 1], header_columns=0,
#                      ax=None, **kwargs):
#     if ax is None:
#         size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
#         fig, ax = plt.subplots(figsize=size)
#         ax.axis('off')

#     mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

#     mpl_table.auto_set_font_size(False)
#     mpl_table.set_fontsize(font_size)

#     for k, cell in six.iteritems(mpl_table._cells):
#         cell.set_edgecolor(edge_color)
#         if k[0] == 0 or k[1] < header_columns:
#             cell.set_text_props(weight='bold', color='w')
#             cell.set_facecolor(header_color)
#         else:
#             cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
#     return ax
#     render_mpl_table(dfreport, header_columns=0, col_width=8.0)
#     plt.savefig("../postfit_derivatives/"+theta+"_summary.png")
#     plt.clf()